In [1]:
from functools import reduce
import vyper
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from vyper.user import Model
from vyper.utils.tools import StatisticalTools as st
from vyper.user import Model
from vyper.user.explorer import DataProfiler
from vyper.utils.tools import StatisticalTools as st
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandas.api.types import is_string_dtype
from openpyxl import Workbook
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import math
import warnings
import string

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.drawing.text import CharacterProperties
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

In [2]:
## EDITS
def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
                     variable_order, other_segment = False, file = 'Profile', exclude = '',
                     PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
    

    profile_data = pd.DataFrame(profile_data)
    
    if (segments is None):
        segments = pd.unique(profile_data[segment_var])
        other_segment = False
        segment_names = str(segments)
    if (not (isinstance(other_segment, bool))):
        raise ValueError('other must be True or False.')
    if (segment_names is None):
        segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    if (len(segments)+other_segment!=len(segment_names)):
        if(len(segments)==len(segment_names)):
            segment_names = [segment_names, 'other']
        else:
            warnings.warn('Incorrect length of names, replacing by segments')
            segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    ## output -> [1 2 0]

    varclass = pd.DataFrame()
    for col in profile_data.columns:
        varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)
        
    # Continuous Variables that were classified as categorical
    varclass[varclass.columns in continuous] = 'continuous'
    if (not (include is None)):
        exclude = varclass.columns[varclass.columns not in include]
    # Variables to exclude
    varclass[varclass.columns in exclude] = 'exclude'
    if (excludeother): 
        varclass[varclass.columns =='Categorical+other'] = 'exclude'
    continuous_var_cuts = dict()

    
    for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
        if not continuous_var_bounds:
            continuous_var_bounds = dict()
        if variable in continuous_var_bounds:
            cut_bins = continuous_var_bounds[variable]
            continuous_var_cuts[variable] = pd.cut(profile_data[variable], bins=cut_bins)
                
        else:
            continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,squash_extremes=False)

    col_order = []
    index_order = []
    ### TODO: array([2, 0, 1]) Always encode 0 as US
    ### Change the US population as BASELINE
    for seg in segments:
        profile_1 = DataProfiler(profile_data[profile_data[segment_var] == seg]).create_profile(number_of_bins = 10,
                                                                            cts_cuts = continuous_var_cuts)
        if (seg == 'Baseline'):
            profile = profile_1
            profile.columns = ["Variable", "Category",  "Baseline_Count","Baseline_Percent"]
            for cols in profile.columns:
                col_order.append(cols)


        else:
            profile_1.columns = ["Variable", "Category",  "Count_"+str(seg), "Percent_"+str(seg)]
            col_order.append("Count_"+str(seg))
            col_order.append("Percent_"+str(seg))
            profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
            profile['Baseline vs ' + "Segment_" + str(seg)] = (profile["Baseline_Percent"]/profile["Percent_"+str(seg)]) * 100
            index_order.append('Baseline vs ' + "Segment_" + str(seg))     

    profile = profile[col_order+index_order]

    for i in profile.columns:

        if ('Count' in i):
            profile[i][profile[i].isna()] = 0
        if ('Percent' in i):
            profile[i][profile[i].isna()] = 0
            
            
    if file:
        import os
        # Make Profile ####
        filesave = file + '.xlsx'
        
        if os.path.exists(filesave):
            os.remove(filesave)
        CreateXLProfile_Snap(profile, savepath = filesave)
        
        if PPT:
            wb = Workbook()

            profile['Category'] = profile['Category'].astype(str)

            all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
            # make new sheet the active sheet we are working on
            all_var_profiling_ws = wb.active

            # remove gridlines from the sheet
            all_var_profiling_ws.sheet_view.showGridLines = False

            # set border line thickness
            thick = Side(border_style="thick", color="000000")
            # medium = Side(border_style="medium", color="000000")
            thin = Side(border_style="thin", color="000000")

            # get max rows and cols of profiling df
            max_rows = profile.shape[0]
            max_cols = profile.shape[1]
            
            input_cols = list(string.ascii_uppercase)

            j = 4
            for z in range(0, len(profile)):
                if (z != 0):
                    if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
                        # create sheet
                        all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
                        # make new sheet the active sheet we are working on
                        all_var_profiling_ws = wb.active

                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                        j = 4
                    else:
                        incol = 2
                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                            incol = incol + 1
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                            if (y % 2 == 1 and y > 2):
                                current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                                incol = incol + 1
                                current_cell.font = Font(bold=True)
                                current_cell.alignment = Alignment(horizontal='center', vertical='center')
                                current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                                current_cell.value = 'Index'
                                if y == max_cols - 1:
                                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                                else:
                                    current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
                incol = 2
                for y in range(len(profile.columns)):
                    current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                    incol = incol + 1
                    current_cell.value = profile.iloc[z, y]
                    if y == 0:
                        current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                    else:
                        current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
                    if (y % 2 == 1 and y > 2):
                        current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                        incol = incol + 1
                        current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
                        if y == max_cols - 1:
                            current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                        else:
                            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

                j = j + 1
            
            filesave = file + 'PPT.xlsx'
        
            if os.path.exists(filesave):
                os.remove(filesave)
            
            wb.save(filesave)
            
    return profile

def CreateXLProfile_Snap(profile, savepath):
    wb = Workbook()

    profile['Category'] = profile['Category'].astype(str)

    # create sheet
    all_var_profiling_ws = wb.create_sheet('profile', 0)

    # make new sheet the active sheet we are working on
    all_var_profiling_ws = wb.active

    # remove gridlines from the sheet
    all_var_profiling_ws.sheet_view.showGridLines = False

    # set border line thickness
    thick = Side(border_style="thick", color="000000")
    # medium = Side(border_style="medium", color="000000")
    thin = Side(border_style="thin", color="000000")

    # get max rows and cols of profiling df
    max_rows = profile.shape[0]
    max_cols = profile.shape[1]

    input_rows = range(4, max_rows + 4)
    input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

    y = 0
    for i in input_cols:
        x = "3"
        x = i + x

        all_var_profiling_ws[x].font = Font(bold=True)
        current_cell = all_var_profiling_ws[x]
        current_cell.alignment = Alignment(horizontal='center', vertical='center')
        current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

        if y == 0:
            current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
        elif y == max_cols - 1:
            current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
        else:
            current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

        all_var_profiling_ws[x] = profile.columns[y]
        y = y + 1
    
    print("Worked till here")
    counter = []
    counter_val = 1
    for index,i in enumerate(input_cols[2::]):
        x = "2"
        x = i + x
        if index%2==1:
            all_var_profiling_ws.merge_cells(f"{counter[-1]}:{x}")
            all_var_profiling_ws[counter[-1]].font = Font(bold=True)
            current_cell = all_var_profiling_ws[counter[-1]]
            current_cell.alignment = Alignment(horizontal='center', vertical='center')
            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
            continue


        if i==input_cols[-1]:
            break
        if i=='D':
            all_var_profiling_ws[x] = "Baseline"
            all_var_profiling_ws.merge_cells(start_row=2, start_column=4, end_row=2, end_column=5)
            counter.append(x)
            continue
        all_var_profiling_ws[x] = 'Segment ' + str(counter_val)
        counter_val+=1
        counter.append(x)
        # all_var_profiling_ws.merge_cells(start_row=2, start_column=5, end_row=2, end_column=5)
        # all_var_profiling_ws.merge_cells(f'{x}:D2')

    
    def style_range(ws, cell_range, border=Border(), fill=None, font=None,alignment=None):
        
        """
        Apply styles to a range of cells as if they were a single cell.

        :param ws:  Excel worksheet instance
        :param range: An excel range to style (e.g. A1:F20)
        :param border: An openpyxl Border
        :param fill: An openpyxl PatternFill or GradientFill
        :param font: An openpyxl Font object
        """

        top = Border(top=border.top)
        left = Border(left=border.left)
        right = Border(right=border.right)
        bottom = Border(bottom=border.bottom)

        first_cell = ws[cell_range.split(":")[0]]
        if alignment:
            ws.merge_cells(cell_range)
            first_cell.alignment = alignment

        rows = ws[cell_range]
        if font:
            first_cell.font = font

        for cell in rows[0]:
            cell.border = cell.border + top
        for cell in rows[-1]:
            cell.border = cell.border + bottom

        for row in rows:
            l = row[0]
            r = row[-1]
            l.border = l.border + left
            r.border = r.border + right
            if fill:
                for c in row:
                    c.fill = fill


    border = Border(top=thick, left=thick, right=thick, bottom=thick)

    for range1 in all_var_profiling_ws.merged_cells.ranges:
        style_range(all_var_profiling_ws, str(range1), border=border)

    

    y = 0
    for i in input_cols:
        z = 0
        for j in input_rows:
            all_var_profiling_ws.row_dimensions[j].height = 25
            x = str(j)
            x = i + x
            # print(x)
            current_cell = all_var_profiling_ws[x]
            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

            if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

            all_var_profiling_ws[x] = profile.iloc[z, y]

            if y == 0:
                current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
            elif y == max_cols - 1:
                current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


            z = z + 1
        y = y + 1
    

    
    if (savepath != ''):
        wb.save(savepath)

In [3]:
# Read in the demonstration Epsilon Response Model Data - Internal Append
# eps_data = pd.read_csv(r"Data/EpsilonResponseModelData_InternalAppend.csv")
eps_data = pd.read_csv('Data/selected_attributes.csv')

# Output length to confirm read in
len(eps_data)

19852

In [4]:
# Read in the Epsilon Data Dictionary we created
epsilon_dict = pd.read_csv('Data/EpsilonDataDictionary.csv')

# Pull list of Selected Attributes from Snowflake Headers
variable_list = epsilon_dict['Snowflake Headers'].tolist()

# Selected Attributes only
new_df = eps_data[variable_list]
new_df

,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,STATE,...,PROPERTY_LOT_SIZE_IN_ACRES_DUPLICATE,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK
0,1,2,36,11209,5503,653,C013,65 89TH ST,BROOKLYN,NY,...,A,64,10,10,360.0,7,7,50002,0,2
1,1,21,36,11427,2143,41,C027,23404 UNION TPKE,QUEENS VLG,NY,...,A,2,10,12,938.0,8,6,1301001,0,3
2,1,8,9,6880,5532,139,C018,113 REGENTS PARK,WESTPORT,CT,...,NaN,50,11,14,455.0,5,7,506003,0,4
3,1,11,36,10028,4456,947,C021,351 E 84TH ST APT 9D,NEW YORK,NY,...,NaN,14,8,12,NaN,4,5,146014,1,0
4,1,11,36,11357,1326,209,C006,620 160TH ST,WHITESTONE,NY,...,A,34,11,11,598.0,9,7,991001,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19847,1,18,34,7463,2301,121,C003,12 MAPLE CT,WALDWICK,NJ,...,C,8,11,12,332.0,9,7,561003,0,16
19848,1,5,36,11507,1123,289,C007,128 NASSAU DR,ALBERTSON,NY,...,A,44,11,14,655.0,7,9,3023003,0,10
19849,1,4,34,7760,1062,632,C005,163 RUMSON RD,RUMSON,NJ,...,NaN,76,10,12,NaN,2,7,8038004,0,20
19850,1,10,36,11209,6757,11,C046,9452 RIDGE BLVD APT 1,BROOKLYN,NY,...,NaN,77,9,9,NaN,7,6,54001,0,0


In [5]:
# Split the data in half just for testing purposes of dummy groups
cutoff = int(len(new_df)/2)
baseline_df = new_df.iloc[:cutoff,:]
customer1_df = new_df.iloc[cutoff:,:]

In [6]:
# Function to combine customer data with baseline and add the Group segment
def append_data(baseline_df, *customer_dfs):
    # Tag baseline group segment
    baseline_df['Group'] = "Baseline"
    append_df = baseline_df
    
    # Start a counter for group segmentation
    customer_count = 1
    
    # Loop over the customer data, adding the group segment then appending
    for customer_df in customer_dfs:
        customer_df['Group'] = f'Customer {str(customer_count)}'
        customer_count += 1
        append_df = append_df.append(customer_df)
        
    append_df.reset_index(inplace=True)
        
    return append_df

In [7]:
final_df = append_data(baseline_df, customer1_df)
len(final_df)

19852

In [8]:
# Function to read in a csv with containing the variable names and the respective bin cutoffs
def continuous_bins(filepath):
    df_continuous = pd.read_csv(filepath)
    df_continuous.dropna(inplace=True)
    
    continuous_bounds = {}
    for index, row in df_continuous.iterrows():
        continuous_bounds[row['Snowflake Field names']] = [float(i) for i in row['Bin'].split(sep=',')]
        
    return continuous_bounds

In [9]:
# Set up variables for snapshot
file_name = '3-30-test'
seg_var = 'Group'
dataset = final_df

# Read in file and set bins
# bin_vars = continuous_bins('Data/bin_dictionary.csv')
bin_vars = continuous_bins('Data/Epsilon_attributes_binning.csv')

In [10]:
# Run Snapshot Report
profile = Snapshot_Profile(profile_data = dataset, segment_var=seg_var, segments = None, segment_names = None, include = None,
                 continuous_var_bounds = bin_vars, variable_order = None, other_segment = False,
                 file = file_name, exclude = [], PPT = True, continuous = [], excludeother = False)
profile

Worked till here


,Variable,Category,Baseline_Count,Baseline_Percent,Count_Customer 1,Percent_Customer 1,Baseline vs Segment_Customer 1
0,index,"(-0.001, 1985.1]",1986.0,20.008060,0.0,0.000000,NaN
1,index,"(1985.1, 3970.2]",1985.0,19.997985,0.0,0.000000,NaN
2,index,"(3970.2, 5955.3]",1985.0,19.997985,0.0,0.000000,NaN
3,index,"(5955.3, 7940.4]",1985.0,19.997985,0.0,0.000000,NaN
4,index,"(7940.4, 9925.5]",1985.0,19.997985,0.0,0.000000,NaN
...,...,...,...,...,...,...,...
6741,VEHICLE_MILEAGE_CODE_2,V,0.0,0.000000,1.0,0.010075,NaN
6742,VEHICLE_MILEAGE_CODE_2,W,0.0,0.000000,1.0,0.010075,NaN
6743,VEHICLE_MILEAGE_CODE_2,X,0.0,0.000000,1.0,0.010075,NaN
6744,VEHICLE_SERVICE_INDICATOR_4,5.0,0.0,0.000000,2.0,0.020149,NaN
